In [42]:
import os
import sys

import numpy as np
import torch
from safetensors.torch import load_model

from models import hvatnet
from utils import creating_dataset, inference_utils

sys.path.insert(1, os.path.realpath(os.path.pardir))


In [45]:
## Init model
device = 'cuda:0'
dtype = torch.float32

MODEL_TYPE = 'hvatnet'
weights = r"logs\test_2_run_fedya\step_1200_loss_0.2587.safetensors"
model_config = hvatnet.Config(n_electrodes=8, n_channels_out=20,
                            n_res_blocks=3, n_blocks_per_layer=3,
                            n_filters=128, kernel_size=3,
                            strides=(2, 2, 2), dilation=2, 
                            small_strides = (2, 2))
model = hvatnet.HVATNetv3(model_config)

load_model(model, weights)

model = model.to(device).to(dtype)

Number of parameters: 4210788


In [46]:
data_paths = dict(datasets=[r"D:\Work\alvi_labs\code\data\processed\dataset_v1_big", 
                           r"D:\Work\alvi_labs\code\data\processed\dataset_v2_blocks"],
                    hand_type = ['left'],
                    human_type = ['fedya_tropin_standart_elbow_left'],
                    test_dataset_list = ['fedya_tropin_standart_elbow_left'])

data_config = creating_dataset.DataConfig(**data_paths)

WINDOW_SIZE = 256
STRIDE = 256

test_dataset = creating_dataset.get_datasets(data_config, transform=None, only_test=True)
test_dataset_list = test_dataset.datasets

all_preds, all_targets =  inference_utils.get_predictions_and_targets(test_dataset_list, model, WINDOW_SIZE, STRIDE)
angles, corrs = inference_utils.calculate_angle_metrics(all_preds, all_targets)

print(f"{MODEL_TYPE=}, {np.mean(angles)=}, {np.mean(corrs)=}")

Getting val datasets
Number of moves: 72 | Dataset: fedya_tropin_standart_elbow_left
Reorder this dataset fedya_tropin_standart_elbow_left True


100%|██████████| 72/72 [00:15<00:00,  4.55it/s]

MODEL_TYPE='hvatnet', np.mean(angles)=14.526435140520737, np.mean(corrs)=0.7965143874070965
